In [2]:
!pip3 install datasets
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U git+https://github.com/lvwerra/trl.git

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

In [20]:
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [23]:
dataset = load_dataset("lucasmccabe-lmi/CodeAlpaca-20k", split="train")

In [28]:
tokenizer.eos_token

'</s>'

### Trainer with DataCollatorForCompletionOnlyLM

Equivalent to PLW = 0.0

In [46]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### Question: {example['instruction'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

response_template = " ### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)


/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:153: UserWarning: No `SFTConfig` passed, using `output_dir=tmp_trainer`.
  warnings.warn(f"No `SFTConfig` passed, using `output_dir={output_dir}`.")
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:302: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/20022 [00:00<?, ? examples/s]

In [47]:
dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 20022
})

In [48]:
masked1 = trainer.data_collator.torch_call([trainer.train_dataset[0]])
masked1

{'input_ids': tensor([[    2, 48134, 15680,    35, 21384,    10,  5043,    14,  1239,    10,
          2167,  8135,     8,  9108,    10,  2167,  4195,   634,   143, 30412,
          5990,     4, 21062, 12337,  3260,    11, 31886,     4, 50118, 22560,
         31652,    35,  3816,   856,  1640,  1178,  3256, 50118,  1437,  1437,
          1437, 49434, 50118,  1437,  1437,  1437, 29072,    10,  2167,  8135,
             8,  9108,    10,  2167,  4195,   634,   143, 30412,  5990, 50118,
          1437,  1437,  1437, 49434, 50118,  1437,  1437,  1437,   671,  3023,
         12606,   176,  2055,   155,  3226,  1178]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]]), 'labels': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
     

In [49]:
masked1['input_ids'][0]

tensor([    2, 48134, 15680,    35, 21384,    10,  5043,    14,  1239,    10,
         2167,  8135,     8,  9108,    10,  2167,  4195,   634,   143, 30412,
         5990,     4, 21062, 12337,  3260,    11, 31886,     4, 50118, 22560,
        31652,    35,  3816,   856,  1640,  1178,  3256, 50118,  1437,  1437,
         1437, 49434, 50118,  1437,  1437,  1437, 29072,    10,  2167,  8135,
            8,  9108,    10,  2167,  4195,   634,   143, 30412,  5990, 50118,
         1437,  1437,  1437, 49434, 50118,  1437,  1437,  1437,   671,  3023,
        12606,   176,  2055,   155,  3226,  1178])

In [50]:
masked1['attention_mask'][0]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1])

In [51]:
masked1['labels'][0]

tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  3816,   856,  1640,  1178,  3256, 50118,  1437,  1437,
         1437, 49434, 50118,  1437,  1437,  1437, 29072,    10,  2167,  8135,
            8,  9108,    10,  2167,  4195,   634,   143, 30412,  5990, 50118,
         1437,  1437,  1437, 49434, 50118,  1437,  1437,  1437,   671,  3023,
        12606,   176,  2055,   155,  3226,  1178])

In [52]:
tokenizer.decode([ 3816,   856,  1640,  1178,  3256, 50118,  1437,  1437,
         1437, 49434, 50118,  1437,  1437,  1437, 29072,    10,  2167,  8135,
            8,  9108,    10,  2167,  4195,   634,   143, 30412,  5990, 50118,
         1437,  1437,  1437, 49434, 50118,  1437,  1437,  1437,   671,  3023,
        12606,   176,  2055,   155,  3226,  1178,     2])

' def f(x):\n    """\n    Takes a specific input and produces a specific output using any mathematical operators\n    """\n    return x**2 + 3*x</s>'

In [53]:
dataset[0]['output']

'def f(x):\n    """\n    Takes a specific input and produces a specific output using any mathematical operators\n    """\n    return x**2 + 3*x'

In [54]:
model.device

device(type='cuda', index=0)

In [55]:
masked1 = masked1.to(model.device)
masked1

{'input_ids': tensor([[    2, 48134, 15680,    35, 21384,    10,  5043,    14,  1239,    10,
          2167,  8135,     8,  9108,    10,  2167,  4195,   634,   143, 30412,
          5990,     4, 21062, 12337,  3260,    11, 31886,     4, 50118, 22560,
         31652,    35,  3816,   856,  1640,  1178,  3256, 50118,  1437,  1437,
          1437, 49434, 50118,  1437,  1437,  1437, 29072,    10,  2167,  8135,
             8,  9108,    10,  2167,  4195,   634,   143, 30412,  5990, 50118,
          1437,  1437,  1437, 49434, 50118,  1437,  1437,  1437,   671,  3023,
         12606,   176,  2055,   155,  3226,  1178]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]], device='cuda:0'), 'labels': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,

In [12]:
trainer.compute_loss(model, masked1)

tensor(2.2431, device='cuda:0', grad_fn=<NllLossBackward0>)

### Trainer with PLW = 1 (default)

In [38]:
trainer2 = SFTTrainer(
    model,
    train_dataset=dataset,
    formatting_func=formatting_prompts_func,
)

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:153: UserWarning: No `SFTConfig` passed, using `output_dir=tmp_trainer`.
  warnings.warn(f"No `SFTConfig` passed, using `output_dir={output_dir}`.")
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:302: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [39]:
masked = trainer2.data_collator.torch_call([trainer.train_dataset[0]])
masked

{'input_ids': tensor([[    2, 48134, 15680,    35, 21384,    10,  5043,    14,  1239,    10,
          2167,  8135,     8,  9108,    10,  2167,  4195,   634,   143, 30412,
          5990,     4, 21062, 12337,  3260,    11, 31886,     4, 50118, 22560,
         31652,    35,  3816,   856,  1640,  1178,  3256, 50118,  1437,  1437,
          1437, 49434, 50118,  1437,  1437,  1437, 29072,    10,  2167,  8135,
             8,  9108,    10,  2167,  4195,   634,   143, 30412,  5990, 50118,
          1437,  1437,  1437, 49434, 50118,  1437,  1437,  1437,   671,  3023,
         12606,   176,  2055,   155,  3226,  1178]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]]), 'labels': tensor([[    2, 48134, 15680,    35, 21384,    10,  5043,    14,  1239,    10,
     

In [40]:
masked = masked.to(model.device)

In [41]:
masked['labels']

tensor([[    2, 48134, 15680,    35, 21384,    10,  5043,    14,  1239,    10,
          2167,  8135,     8,  9108,    10,  2167,  4195,   634,   143, 30412,
          5990,     4, 21062, 12337,  3260,    11, 31886,     4, 50118, 22560,
         31652,    35,  3816,   856,  1640,  1178,  3256, 50118,  1437,  1437,
          1437, 49434, 50118,  1437,  1437,  1437, 29072,    10,  2167,  8135,
             8,  9108,    10,  2167,  4195,   634,   143, 30412,  5990, 50118,
          1437,  1437,  1437, 49434, 50118,  1437,  1437,  1437,   671,  3023,
         12606,   176,  2055,   155,  3226,  1178]], device='cuda:0')

In [42]:
loss, outputs = trainer2.compute_loss(model, masked, return_outputs=True)

In [43]:
loss

tensor(3.1653, device='cuda:0', grad_fn=<NllLossBackward0>)

### Custom Trainer for PLW

Tests:
1. PLW = 1.0 => Should give the same output as default huggingface SFTTrainer
2. PLW = 0.0 => Should give the same output as SFTTrainer with DataCollatorForCompletionOnlyLM

In [13]:
import torch
import numpy as np
import warnings

In [14]:
from typing import Any, Dict, List, Literal, Optional, Tuple, Union

In [15]:
from transformers import DataCollatorForLanguageModeling

In [58]:
class SFTTrainerWithPLW(SFTTrainer):
    def __init__(self, *args, plw=1.0, response_template_tokens, **kwargs):
        super().__init__(*args, **kwargs)
        self.plw = plw
        self.response_template_tokens = response_template_tokens

    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
        logits = outputs.get("logits")
        labels = inputs.pop("labels")

        # Initialize the weights matrix with ones, keeping the same shape as labels
        weights = torch.ones_like(labels, dtype=torch.float32)

        batch_size = labels.shape[0]
        labels_length = labels.shape[1]
        response_tokens_length = len(self.response_template_tokens)

        # Iterate through each example in the batch to find the completion_start_idx
        for batch_idx in range(batch_size):
            completion_start_idx = None

            # Search for response_tokens in labels for the current batch element
            for i in range(labels_length - response_tokens_length + 1):
                if labels[batch_idx, i:i + response_tokens_length].tolist() == self.response_template_tokens:
                    completion_start_idx = i + response_tokens_length - 1
                    break

            # If we found completion_start_idx, modify weights for the current batch element
            if completion_start_idx is not None:
                weights[batch_idx, :completion_start_idx + 1] = self.plw

        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        shift_weights = weights[..., 1:].contiguous()

        shift_labels = shift_labels.to(shift_logits.device)
        shift_weights = shift_weights.to(shift_logits.device)

        # per-token losses
        loss_fct = torch.nn.CrossEntropyLoss(reduction="none")
        token_losses = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), 
                                shift_labels.view(-1))

        # Compute weighted average of losses
        loss = (token_losses.float() @ shift_weights.view(-1).float()) / shift_weights.sum()
        return (loss, outputs) if return_outputs else loss

In [59]:
response_template = " ### Answer:"
response_template_tokens = tokenizer.encode(response_template)[2:]
response_template_tokens

[31652, 35]

In [60]:
trainer3 = SFTTrainerWithPLW(
    model,
    train_dataset=dataset,
    formatting_func=formatting_prompts_func,
    response_template_tokens=response_template_tokens,
    plw=0.0
)

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:153: UserWarning: No `SFTConfig` passed, using `output_dir=tmp_trainer`.
  warnings.warn(f"No `SFTConfig` passed, using `output_dir={output_dir}`.")
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:302: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [45]:
masked = trainer3.data_collator.torch_call([trainer.train_dataset[0]])
masked

{'input_ids': tensor([[    2, 48134, 15680,    35, 21384,    10,  5043,    14,  1239,    10,
          2167,  8135,     8,  9108,    10,  2167,  4195,   634,   143, 30412,
          5990,     4, 21062, 12337,  3260,    11, 31886,     4, 50118, 22560,
         31652,    35,  3816,   856,  1640,  1178,  3256, 50118,  1437,  1437,
          1437, 49434, 50118,  1437,  1437,  1437, 29072,    10,  2167,  8135,
             8,  9108,    10,  2167,  4195,   634,   143, 30412,  5990, 50118,
          1437,  1437,  1437, 49434, 50118,  1437,  1437,  1437,   671,  3023,
         12606,   176,  2055,   155,  3226,  1178]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]]), 'labels': tensor([[    2, 48134, 15680,    35, 21384,    10,  5043,    14,  1239,    10,
     

In [46]:
masked = masked.to(model.device)

In [47]:
masked['labels']

tensor([[    2, 48134, 15680,    35, 21384,    10,  5043,    14,  1239,    10,
          2167,  8135,     8,  9108,    10,  2167,  4195,   634,   143, 30412,
          5990,     4, 21062, 12337,  3260,    11, 31886,     4, 50118, 22560,
         31652,    35,  3816,   856,  1640,  1178,  3256, 50118,  1437,  1437,
          1437, 49434, 50118,  1437,  1437,  1437, 29072,    10,  2167,  8135,
             8,  9108,    10,  2167,  4195,   634,   143, 30412,  5990, 50118,
          1437,  1437,  1437, 49434, 50118,  1437,  1437,  1437,   671,  3023,
         12606,   176,  2055,   155,  3226,  1178]], device='cuda:0')

In [48]:
loss, outputs = trainer3.compute_loss(model, masked, return_outputs=True)

In [49]:
loss

tensor(2.2431, device='cuda:0', grad_fn=<DivBackward0>)

### PLW With Custom Data Collator

In [79]:
import torch
import numpy as np

In [80]:
from transformers import DataCollatorForLanguageModeling
from typing import TYPE_CHECKING, Any, Callable, Dict, List, Optional, Tuple, Union

In [81]:
class DataCollatorForPLW(DataCollatorForLanguageModeling):

    def __init__(
        self,
        response_template: Union[str, List[int]],
        instruction_template: Optional[Union[str, List[int]]] = None,
        *args,
        mlm: bool = False,
        ignore_index: int = -100,
        padding_free: bool = False,
        **kwargs,
    ):
        super().__init__(*args, mlm=mlm, **kwargs)

        self.instruction_template = instruction_template
        if isinstance(instruction_template, str):
            # The user provides a string, must tokenize
            self.instruction_token_ids = self.tokenizer.encode(self.instruction_template, add_special_tokens=False)
        else:
            # The user already provides the token ids
            self.instruction_token_ids = instruction_template

        self.response_template = response_template
        if isinstance(response_template, str):
            # The user provides a string, must tokenize
            self.response_token_ids = self.tokenizer.encode(self.response_template, add_special_tokens=False)
        else:
            # The user already provides the token ids
            self.response_token_ids = response_template

        if not self.mlm and self.instruction_template and self.tokenizer.pad_token_id == self.tokenizer.eos_token_id:
            warnings.warn(
                "The pad_token_id and eos_token_id values of this tokenizer are identical. "
                "If you are planning for multi-turn training, "
                "it can result in the model continuously generating questions and answers without eos token. "
                "To avoid this, set the pad_token_id to a different value."
            )

        self.ignore_index = ignore_index
        self.padding_free = padding_free

    def torch_call(self, examples: List[Union[List[int], Any, Dict[str, Any]]]) -> Dict[str, Any]:
        batch = super().torch_call(examples)
        
        # Initialize the "weights" column to be all ones
        batch_size = batch["labels"].shape[0]
        seq_len = batch["labels"].shape[1]
        batch["prompt_mask"] = torch.zeros((batch_size, seq_len), dtype=torch.float32)


        if self.instruction_template is None:
            for i in range(len(examples)):
                response_token_ids_start_idx = None

                for idx in np.where(batch["labels"][i] == self.response_token_ids[0])[0]:
                    # `response_token_ids` is `'### Response:\n'`, here we are just making sure that the token IDs match
                    if (
                        self.response_token_ids
                        == batch["labels"][i][idx : idx + len(self.response_token_ids)].tolist()
                    ):
                        response_token_ids_start_idx = idx

                if response_token_ids_start_idx is None:
                    warnings.warn(
                        f"Could not find response key `{self.response_template}` in the "
                        f'following instance: {self.tokenizer.decode(batch["input_ids"][i])} '
                        f"This instance will be ignored in loss calculation. "
                        f"Note, if this happens often, consider increasing the `max_seq_length`."
                    )
                    batch["prompt_mask"][i, :] = 1
                else:
                    response_token_ids_end_idx = response_token_ids_start_idx + len(self.response_token_ids)

                    # Make pytorch loss function ignore all tokens up through the end of the response key
                    batch["prompt_mask"][i, :response_token_ids_end_idx] = 1

        else:
            for i in range(len(examples)):
                response_token_ids_idxs = []
                human_token_ids_idxs = []

                for assistant_idx in np.where(batch["labels"][i] == self.response_token_ids[0])[0]:
                    # find the indexes of the start of a response.
                    if (
                        self.response_token_ids
                        == batch["labels"][i][assistant_idx : assistant_idx + len(self.response_token_ids)].tolist()
                    ):
                        response_token_ids_idxs.append(assistant_idx + len(self.response_token_ids))

                if len(response_token_ids_idxs) == 0:
                    warnings.warn(
                        f"Could not find response key `{self.response_template}` in the "
                        f'following instance: {self.tokenizer.decode(batch["input_ids"][i])} '
                        f"This instance will be ignored in loss calculation. "
                        f"Note, if this happens often, consider increasing the `max_seq_length`."
                    )
                    batch["prompt_mask"][i, :] = 1

                human_token_ids = self.instruction_token_ids
                for human_idx in np.where(batch["labels"][i] == human_token_ids[0])[0]:
                    # find the indexes of the start of a human answer.
                    if human_token_ids == batch["labels"][i][human_idx : human_idx + len(human_token_ids)].tolist():
                        human_token_ids_idxs.append(human_idx)

                if len(human_token_ids_idxs) == 0:
                    warnings.warn(
                        f"Could not find instruction key `{self.instruction_template}` in the "
                        f'following instance: {self.tokenizer.decode(batch["input_ids"][i])} '
                        f"This instance will be ignored in loss calculation. "
                        f"Note, if this happens often, consider increasing the `max_seq_length`."
                    )
                    batch["prompt_mask"][i, :] = 1

                if (
                    len(human_token_ids_idxs) > 0
                    and len(response_token_ids_idxs) > 0
                    and human_token_ids_idxs[0] > response_token_ids_idxs[0]
                ):
                    human_token_ids_idxs = [0] + human_token_ids_idxs

                for idx, (start, end) in enumerate(zip(human_token_ids_idxs, response_token_ids_idxs)):
                    # Make pytorch loss function ignore all non response tokens
                    if idx != 0:
                        batch["prompt_mask"][i, start:end] = 1
                    else:
                        batch["prompt_mask"][i, :end] = 1

                if len(response_token_ids_idxs) < len(human_token_ids_idxs):
                    batch["prompt_mask"][i, human_token_ids_idxs[-1] :] = 1

        if self.padding_free:
            # remove padding, `attention_mask` and add `position_ids`
            attn_mask = batch.pop("attention_mask")
            batch["input_ids"] = batch["input_ids"][attn_mask.bool()].unsqueeze(0)
            batch["position_ids"] = attn_mask.cumsum(1)[attn_mask.bool()].unsqueeze(0) - 1
            batch["labels"] = batch["labels"][attn_mask.bool()].unsqueeze(0)
            batch["prompt_mask"] = batch["prompt_mask"][attn_mask.bool()].unsqueeze(0)
            batch["labels"][batch["position_ids"] == 0] = self.ignore_index
            batch["prompt_mask"][batch["position_ids"] == 0] = 1
            
        return batch

In [141]:
class SFTTrainerWithPLWCollator(SFTTrainer):
    def __init__(self, *args, plw=1.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.plw = plw

    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
        logits = outputs.get("logits")
        labels = inputs.pop("labels")
        
        prompt_mask = inputs["prompt_mask"]
        weights = torch.where(prompt_mask == 1, self.plw, torch.tensor(1.0, device=prompt_mask.device))
        
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        shift_weights = weights[..., 1:].contiguous()

        shift_labels = shift_labels.to(shift_logits.device)
        shift_weights = shift_weights.to(shift_logits.device)

        # per-token losses
        loss_fct = torch.nn.CrossEntropyLoss(reduction="none")
        token_losses = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), 
                                shift_labels.view(-1))

        # Compute weighted average of losses
        loss = (token_losses.float() @ shift_weights.view(-1).float()) / shift_weights.sum()
        return (loss, outputs) if return_outputs else loss

In [133]:
response_template = " ### Answer:"

In [188]:
response_template_token_ids = tokenizer.encode(response_template)[1:]
response_template_token_ids

[22560, 31652, 35]

In [189]:
plw_collator = DataCollatorForPLW(response_template_token_ids, tokenizer=tokenizer)

In [252]:
# def preprocess_logits_for_metrics(logits, labels):
#     # get predictions
#     token_preds = logits.argmax(-1)[..., :-1]

#     # compute per-token losses
#     loss_fct = torch.nn.CrossEntropyLoss(reduction="none")
#     shift_logits = logits[..., :-1, :].contiguous()
#     shift_labels = labels[..., 1:].contiguous()
#     token_losses = loss_fct(shift_logits.transpose(1, 2), shift_labels)

#     # pass predictions and losses to compute_metrics function (above)
#     predictions = (token_preds, token_losses)
#     return predictions

In [253]:
# def prepare_compute_metrics(response_template_token_ids):
    
#     def compute_metrics(data):
#         token_preds, token_losses = data.predictions
        
#         label_ids = data.label_ids
#         batch_size = label_ids.shape[0]
#         labels_length = label_ids.shape[1]
#         response_tokens_length = len(response_template_token_ids)
        
#         prompt_mask = torch.zeros(batch_size, labels_length)
#         completion_mask = torch.zeros(batch_size, labels_length)
        
#         for batch_idx in range(batch_size):
#             completion_start_idx = None

#             # Search for response_tokens in labels for the current batch element
#             for i in range(labels_length - response_tokens_length + 1):
#                 if label_ids[batch_idx, i:i + response_tokens_length].tolist() == response_template_token_ids:
#                     completion_start_idx = i + response_tokens_length
#                     break

#             if completion_start_idx is not None:
#                 completion_mask[batch_idx, completion_start_idx:] = 1
#                 prompt_mask[batch_idx, :completion_start_idx] = 1
        
#         # shift labels and masks
#         labels = label_ids[..., 1:]
#         shift_prompt_mask = prompt_mask[..., 1:]
#         shift_comp_mask = completion_mask[..., 1:]
        
#         # average both losses (prompt and completion) over their respective tokens
#         token_losses = torch.tensor(token_losses)
#         prompt_loss = token_losses.reshape(-1) @ shift_prompt_mask.reshape(-1) / shift_prompt_mask.sum()
#         completion_loss = token_losses.reshape(-1) @ shift_comp_mask.reshape(-1) / shift_comp_mask.sum()
        
#         return {
#             'comp_loss': completion_loss,
#             'prompt_loss': prompt_loss,
#         }
    
#     return compute_metrics

In [254]:
trainer4 = SFTTrainerWithPLWCollator(
    model,
    train_dataset=dataset,
    formatting_func=formatting_prompts_func,
    data_collator=plw_collator,
#     compute_metrics=prepare_compute_metrics(response_template_token_ids),
#     preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    plw=0
)

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:153: UserWarning: No `SFTConfig` passed, using `output_dir=tmp_trainer`.
  warnings.warn(f"No `SFTConfig` passed, using `output_dir={output_dir}`.")
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:302: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [255]:
masked = trainer4.data_collator.torch_call([trainer.train_dataset[0]])

In [256]:
masked = masked.to(model.device)

In [257]:
loss, outputs = trainer4.compute_loss(model, masked, return_outputs=True)

In [258]:
loss

tensor(2.2431, device='cuda:0', grad_fn=<DivBackward0>)